# PageRank на Spark RDD

In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


1. Прочитайте граф из файла `/user/pavel.klemenkov/lectures/lecture01/data/graph.tsv`
2. Создайте RDD, в которой граф будет представлен парами вершин
3. Убедитесь, что граф совпадает с рисунком на доске

In [3]:
raw_graph = sc.textFile("/user/pavel.klemenkov/lectures/lecture01/data/graph.tsv")

In [5]:
graph = raw_graph.map(lambda x: x.split("\t"))

In [6]:
graph.collect()

[['A', 'B'],
 ['A', 'C'],
 ['B', 'D'],
 ['C', 'A'],
 ['C', 'B'],
 ['C', 'D'],
 ['D', 'C']]

Создайте RDD, с первоначальными pagerank всех уникальных вершин

In [9]:
rank = graph.groupByKey().map(lambda x: (x[0], 1))

In [10]:
rank.collect()

[('C', 1), ('D', 1), ('A', 1), ('B', 1)]

Создайте RDD, которая берет RDD с вершинами, объединяет ее с RDD с pagerank. В результате должна получится PairRDD, где ключ - это уникальная вершина, а значение - это все вершины, на которые она ссылаются и ее текущий pagerank

In [11]:
links = graph.groupByKey().join(rank)

In [12]:
links.collect()

[('A', (<pyspark.resultiterable.ResultIterable at 0x7fa0e5962f28>, 1)),
 ('B', (<pyspark.resultiterable.ResultIterable at 0x7fa0e5962f98>, 1)),
 ('C', (<pyspark.resultiterable.ResultIterable at 0x7fa0e5962f60>, 1)),
 ('D', (<pyspark.resultiterable.ResultIterable at 0x7fa0e5962fd0>, 1))]

Реализуйте функцию, которая подсчитывает новые значения pagerank для вершин, на которые ссылается вершина из RDD из предыдущего шага. Фугкция должна возвращать вершины и их обновленный pagerank.

In [20]:
def computeContribs(neighbours, rank):
    neighbours = list(neighbours)
    for n in neighbours:
        return (n, rank / len(neighbours))

Обновите RDD с pagerank значениями, посчитанными с помощью функции из предыдущей ячейки

In [21]:
links.map(lambda x: computeContribs(x[1][0], x[1][1])).collect()

[('B', 0.5), ('D', 1.0), ('A', 0.3333333333333333), ('C', 1.0)]

Напишите цикл, который проводит несколько итераций вычисленя pagerank и на каждой печатает номер итерации и текущие pagerank

In [ ]:
## Ваш код здесь

In [ ]:
sc.stop()